In [6]:
# датасеты
from datasets import load_dataset
# время обработки задач
from tqdm.auto import tqdm
# пути в операционке (заменить на OS)
import os
from pathlib import Path

In [7]:
dataset = load_dataset('oscar', 'unshuffled_deduplicated_el')

Generating train split:   0%|          | 0/6521169 [00:00<?, ? examples/s]

C:\Users\m092\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [24]:
dataset["train"][999]['text']

'Πολλοί ιδιοκτήτες σκύλων θα ήθελαν να απολαύσουν την γυμναστική τους με το αγαπημένο τους κατοικίδιο. Οι πιο συνηθισμένοι τρόποι για να γίνει αυτό είναι ο περίπατος, το τρέξιμο η κάνοντας ποδήλατο με το σκυλάκι να τρέχει πίσω μας.\nΩστόσο, ο κύριος Nic Bello ψάχνοντας κάτι πιο ξεκούραστο ξεκίνησε να διδάσκει το μικρό του φίλο “Pancho”, yoga. Το μικρό και ευερέθιστο Τσιουάουα όπως θα δείτε, όχι μόνο ανταποκρίνεται, δείχνει και να το διασκεδάζει. Σίγουρα μερικά τεντώματα και βαθιές εισπνοές θα το βοηθήσουν να χαλαρώσει.\nΤο “ζευγάρι” έχει πλέον κυκλοφορήσει μια σειρά από doga βίντεο όπου παρουσιάζουν κάποιες πιο περίπλοκες κινήσεις.'

In [8]:
el_path = './data/text/oscar_el/'
os.makedirs(el_path, exist_ok=True)

In [9]:
text_data = []
file_count = 0
for sample in dataset["train"]:
    sample_text = sample['text'].replace('\n', '')
    text_data.append(sample_text)
    
    if len(text_data) == 10_000:
        # Save tokens to a file every 10,000 samples
        with open(os.path.join(el_path, f'text_{file_count}.txt'), 'w', encoding='utf-8') as file:
            file.write('\n'.join(text_data))
        text_data = []
        file_count += 1

# че-то зачем-то ещё раз сохранили
# Save any remaining tokens
if text_data:
    with open(os.path.join(el_path, f'text_{file_count}.txt'), 'w', encoding='utf-8') as file:
        file.write('\n'.join(text_data))

In [28]:
# tokenization_files

In [10]:
# сохранили тексты, молодцы, а вот пути до них:
tokenization_files = [str(x) for x in Path('./data/text/oscar_el').glob('**/*.txt')]
# tokenization_files

In [12]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=tokenization_files[:200], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [13]:
os.mkdir('./lil_el_model')
tokenizer.save_model('lil_el_model')

['lil_el_model\\vocab.json', 'lil_el_model\\merges.txt']

In [30]:
from transformers import RobertaTokenizerFast

In [31]:
# и вот оно может работать, ура
smart_tokenizer = RobertaTokenizerFast.from_pretrained('lil_el_model')

In [35]:
example_phrase = "Καλημερα"
# example_phrase = "Η Γερμανία κατέληξε τελικά σε συμφωνία με την Ιταλία για την επιστροφή μεταναστών που είχαν ζητήσει άσυλο στη χώρα αυτή"

In [36]:
# smart_tokenizer(example_phrase, padding="max_length", max_length=100)
smart_tokenizer(example_phrase)

{'input_ids': [0, 13571, 16578, 2], 'attention_mask': [1, 1, 1, 1]}

Теперь обучим на полном датасете

MLM

In [45]:
import torch
from copy import deepcopy
from tqdm import tqdm

In [42]:
def mlm(tensor):
    rand = torch.rand(tensor.shape)
    # mask_arr = (rand < 0.15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4
    return tensor

In [43]:
# tokenization_files # всё ещё все файлы
input_ids = []
mask = []
labels = []

In [44]:
for path in tqdm(tokenization_files[:50]):
    with open(path, 'r', encoding="utf-8") as file:
        lines = file.read().split('\n')
    sample = smart_tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt') # truncation=true обрезать слишком длинные строки
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone())) # .detach().clone() — как deepcopy мы не хотим портить настоящий список с id, т. к. он нужен в labels

  0%|          | 0/50 [00:00<?, ?it/s]

In [52]:
# # можно было и сразу нормально записывать, нет?..
# input_ids = torch.cat(input_ids)
# mask = torch.cat(mask)
# labels = torch.cat(labels)

In [56]:
# torch.save(tensor_name, 'tensor_file.pt') # можно схранить


In [57]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [58]:
dataset = Dataset({'input_ids': input_ids, 'attention_mask': mask, 'labels': labels})

In [59]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [63]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

config = RobertaConfig(
    vocab_size=30_522,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)
model = RobertaForMaskedLM(config)

In [64]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [65]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

C:\Users\m092\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [2]:
from tqdm import tqdm

In [3]:
epochs = 200

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

NameError: name 'loader' is not defined

In [ ]:
model.save_pretrained('./lil_el_model')